In [1]:
import xarray as xr
import nowpp as now
import xscale.signal.fitting as xfit
import pandas as pd
import os
import subprocess

## Starting a PBS Cluster with dask.distributed

In [3]:
cluster, client = now.raijin_cluster(cores=14, memory='64GB',  processes=1)

/g/data1/e14/gs9353/miniconda3/envs/now-postprocess/lib/python3.6/site-packages/distributed/deploy/local.py:138: UserWarning: diagnostics_port has been deprecated. Please use `dashboard_address=` instead
  "diagnostics_port has been deprecated. "


In [4]:
print(cluster)

PBSCluster(cores=0, memory=0 B, workers=0/0, jobs=0/0)


Dask.job_queue will launched PBS jobs using the following header:

In [5]:
print(cluster.job_script())

#!/bin/bash

#!/usr/bin/env bash
#PBS -P e14
#PBS -N dask-worker
#PBS -q expressbw
#PBS -l ncpus=14
#PBS -l mem=64GB
#PBS -l walltime=01:00:00
JOB_ID=${PBS_JOBID%.*}



/g/data1/e14/gs9353/miniconda3/envs/now-postprocess/bin/python -m distributed.cli.dask_worker tcp://10.9.105.5:41218 --nthreads 14 --memory-limit 64.00GB --name dask-worker--${JOB_ID}-- --death-timeout 60



In [6]:
client

<Client: scheduler='tcp://10.9.105.5:41218' processes=0 cores=0>

Now, you can ask for ressources by launching a defined number of jobs on Raijin with `cluster.scale`. The total ressources available will be equal to those defined within the cluster times the scaling factor. 

In [7]:
cluster.scale(1)

You can check the status of your jobs with the following command:

In [3]:
!qstat -u rm6294

/bin/sh: myjobs: command not found


In [4]:
subprocess.check_output(['qstat', '-u', os.environ['USER']])

b''

## Choosing a configuration file

In [14]:
config_file = '/g/data1/e14/gs9353/CONFIG/ecl_study.ini'

## Opend NEMO grids and some postprocess files

In [16]:
gridU = now.io.open_nemo_griddata_from_zarr(config_file, grid='U')
gridU_mean = now.io.open_postprocess(config_file, 'ECL_NEMO_1990-2008_1d_grid_U_2D_mean.nc', 
                                     chunks={'simulation': 1})
gridV = now.io.open_nemo_griddata_from_zarr(config_file, grid='V')
gridV_mean = now.io.open_postprocess(config_file, 'ECL_NEMO_1990-2008_1d_grid_V_2D_mean.nc', 
                                     chunks={'simulation': 1})
gridT = now.io.open_nemo_griddata_from_zarr(config_file, grid='T')
gridT_mean = now.io.open_postprocess(config_file, 'ECL_NEMO_1990-2008_1d_grid_T_2D_mean.nc', 
                                     chunks={'simulation': 1})

In [18]:
ux_mean = gridU_mean.uos
taux_mean = gridU_mean.tauuo
vy_mean = gridV_mean.vos 
tauy_mean = gridV_mean.tauvo
ssh_mean = gridT_mean.zos
temperature_mean = gridT_mean.tos
salinity_mean = gridT_mean.sos

In [19]:
# Horizontal velocities
ux = gridU.uos - gridU_mean.uos.persist()
vy = gridV.vos - gridV_mean.vos.persist()
# Wind stress
taux = gridU.tauuo - gridU_mean.tauuo.persist()
tauy = gridV.tauvo - gridV_mean.tauvo.persist()
# Sea level
sla = gridT.zos - gridT_mean.zos.persist()
# Temperature and Salinity
temperature = gridT.tos - temperature_mean.persist()
salinity = gridT.sos - salinity_mean.persist()
# Mixed Layer Depth
h = gridT.mld_dt02
# Heat Fluxes, water flux
qnet = gridT.tohfls
ep = gridT.wfo

Worker tcp://10.9.67.15:39233 restart in Job 20315. This can be due to memory issue.


## Compute geostrophic and ageostrophic velocities
Here, we estimate geostrophic velocities from sea level anomalies. Ageostrophic velocities are simply obtained by subtracting the geostrophic velocities to the absolute velocities.

In [20]:
ux_geo, vy_geo = now.physics.compute_geostrophic_velocities(config_file, sla)
ux_ageo = ux - ux_geo
vy_ageo = vy - vy_geo

In [21]:
ux_geo_mean, vy_geo_mean = now.physics.compute_geostrophic_velocities(config_file, ssh_mean)
ux_ageo_mean = ux_mean - ux_geo_mean
vy_ageo_mean = vy_mean - vy_geo_mean

## Computing Mean Eddy Kinetic Energy

\begin{equation}
MKE = \frac{1}{2} \left(\overline{u}^2 + \overline{v}^2 \right)
\end{equation}

In [18]:
mke_total = now.physics.kinetic_energy(config_file, ux_mean, vy_mean)
mke_geo = now.physics.kinetic_energy(config_file, ux_geo_mean, vy_geo_mean)
mke_ageo = now.physics.kinetic_energy(config_file, ux_ageo_mean, vy_ageo_mean)

In [19]:
mke = xr.Dataset({'mke': mke_total, 
                  'mke_geo': mke_geo, 
                  'mke_ageo': mke_ageo})

In [20]:
now.io.to_postprocess(mke, config_file, 'ECL_NEMO_1990-2008_1d_Mean_Kinetic_Energy.nc', type='netcdf')

## Computing Mean Wind Work

In [21]:
mww_total = now.physics.wind_energy_transfer(config_file, ux_mean, vy_mean, taux_mean, tauy_mean)
mww_geo = now.physics.wind_energy_transfer(config_file, ux_geo_mean, vy_geo_mean, taux_mean, tauy_mean)
mww_ageo = now.physics.wind_energy_transfer(config_file, ux_ageo_mean, vy_ageo_mean, taux_mean, tauy_mean)

In [22]:
mww = xr.Dataset({'mww': mww_total, 
                  'mww_geo': mww_geo, 
                  'mww_ageo': mww_ageo})

In [23]:
now.io.to_postprocess(mww.assign_coords(nav_lon=sla['nav_lon'], nav_lat=sla['nav_lat']), 
                      config_file, 'ECL_NEMO_1990-2008_1d_Mean_Wind_Work.nc', type='netcdf')

## Computing Eddy Kinetic Energy

In [18]:
eke_total = now.physics.kinetic_energy(config_file, ux, vy).mean('time_counter')
eke_geo = now.physics.kinetic_energy(config_file, ux_geo - ux_geo_mean, vy_geo_mean).mean('time_counter')
eke_ageo = now.physics.kinetic_energy(config_file, ux_ageo, vy_ageo).mean('time_counter')

In [19]:
eke = xr.Dataset({'eke': eke_total.compute(), 
                  'eke_ageo': eke_ageo.compute(), 
                  'eke_geo': eke_geo.compute()})

In [20]:
now.io.to_postprocess(eke, config_file, 'ECL_NEMO_1990-2008_1d_Eddy_Kinetic_Energy.nc', type='netcdf')

## Compute the Eddy Wind Work

In [26]:
eww_total = now.physics.wind_energy_transfer(config_file, ux, vy, taux, tauy).mean('time_counter')
eww_geo = now.physics.wind_energy_transfer(config_file, ux_geo,vy_geo, taux, tauy).mean('time_counter')
eww_ageo = now.physics.wind_energy_transfer(config_file, ux_ageo,vy_ageo, taux, tauy).mean('time_counter')

In [29]:
eddy_wind_work = xr.Dataset({'eww': eww_total.compute(),
                             'eww_geo': eww_geo.compute(),
                             'eww_ageo': eww_ageo.compute()})

In [31]:
now.io.to_postprocess(eddy_wind_work, config_file, 'ECL_NEMO_1990-2008_1d_Eddy_Wind_Work.nc', type='netcdf')

# Eddy Kinetic Energy budget
TODO

# Salinity budget

$ R_S = - \langle u_g' \cdot \nabla S'\rangle  
        - \langle u_a' \cdot \nabla S'\rangle 
        - \langle u_g \rangle \cdot \nabla \langle S \rangle 
        - \langle u_a \rangle \cdot \nabla \langle S \rangle 
        + \langle \frac{(E - P) S}{h}\rangle $

* Eddy salinity fluxes due to geostrophic and ageostrophic motions
* Mean salinity fluxes induced by mean geostrophic and ageostrophic motions
* Water fluxes at surface that freshen or salten

In [21]:
esf_geo = now.physics.tracer_flux(config_file, ux_geo, vy_geo, salinity).mean('time_counter').compute()
esf_ageo = now.physics.tracer_flux(config_file, ux_ageo, vy_ageo, salinity).mean('time_counter').compute()
msf_geo = now.physics.tracer_flux(config_file, ux_geo_mean, vy_geo_mean, salinity_mean).compute()
msf_ageo = now.physics.tracer_flux(config_file, ux_ageo_mean, vy_ageo_mean, salinity_mean).compute()
ssf = (ep * (salinity + salinity_mean) / h).mean('time_counter').compute()
residual = - esf_geo - esf_ageo - msf_geo - msf_ageo + ssf

In [23]:
salinity_budget = xr.Dataset({'msf_geo': msf_geo,
                              'msf_ageo': msf_ageo,
                              'esf_geo': esf_geo,
                              'esf_ageo': esf_ageo,
                              'ssf': ssf,
                              'residual': residual})

In [24]:
now.io.to_postprocess(salinity_budget.assign_coords(nav_lon=sla['nav_lon'], nav_lat=sla['nav_lat']), 
                      config_file, 'ECL_NEMO_1990-2008_1d_SSS_budget.nc', type='netcdf')

# Temperature budget

$ R_T = - \langle u_g' \cdot \nabla T'\rangle  
        - \langle u_a' \cdot \nabla T'\rangle 
        - \langle u_g \rangle \cdot \nabla \langle T \rangle 
        - \langle u_a \rangle \cdot \nabla \langle T \rangle 
        + \langle \frac{Q_{net}}{\rho_0 c_p h}\rangle $

* Eddy heat fluxes due to geostrophic and ageostrophic motions
* Mean heat fluxes induced by mean geostrophic and ageostrophic motions
* Atmospheric total heat fluxes at surface

In [22]:
rho0 = 1025
cp = 4180

In [27]:
ehf_geo = now.physics.tracer_flux.compute_tracer_flux(config_file, ux_geo, vy_geo, temperature).mean('time_counter').compute()
ehf_ageo = now.physics.tracer_flux.compute_tracer_flux(config_file, ux_ageo, vy_ageo, temperature).mean('time_counter').compute()
mhf_geo = now.physics.tracer_flux.compute_tracer_flux(config_file, ux_geo_mean, vy_geo_mean, temperature_mean).compute()
mhf_ageo = now.physics.tracer_flux.compute_tracer_flux(config_file, ux_ageo_mean, vy_ageo_mean, temperature_mean).compute()
shf = (qnet / (rho0 * cp * h)).mean('time_counter').compute()
residual = - ehf_geo - ehf_ageo - mhf_geo - mhf_ageo + shf

In [28]:
temperature_budget = xr.Dataset({'mhf_geo': mhf_geo,
                                 'mhf_ageo': mhf_ageo,
                                 'ehf_geo': ehf_geo,
                                 'ehf_ageo': ehf_ageo,
                                 'shf': shf,
                                 'residual': residual})

In [29]:
now.io.to_postprocess(temperature_budget.assign_coords(nav_lon=sla['nav_lon'], nav_lat=sla['nav_lat']), 
                      config_file, 'ECL_NEMO_1990-2008_1d_SST_budget.nc', type='netcdf')

**Don't forget to close the client and the cluster to avoid Raijin debiting resources** 

In [17]:
client.close()
cluster.close()

# Ekman pumping
TODO